In [1]:
import os
import re
import json
import codecs
import locale
import requests
import datetime

import tabula

import pandas as pd


In [2]:
#----------------------------------------------------------------
# Configurations section
#----------------------------------------------------------------

# Url of the pdf file to download:
url_region_pdf = "http://www.salute.gov.it/imgs/C_17_notizie_4677_0_file.pdf"
                  
#----------------------------------------------------------------
#
#----------------------------------------------------------------
now = datetime.datetime.now()
sample_date = now.strftime("%d/%m/%Y")

locale.setlocale(locale.LC_ALL, 'it_IT.UTF-8')
ok_statuses = [200, 201, 202]
data_file_path = os.path.join("..","data")
#data_file_path = os.path.join(os.sep,"tmp")

pdf_file_name = os.path.join(os.sep, "tmp", "temp_data_file.pdf")
it_data_file = os.path.join(data_file_path, "virus-it.csv")
it_tmp_data_file = os.path.join(data_file_path, "virus-it-{dt}.csv".format(dt=now.strftime("%Y%m%d")))

lomb_data_file = os.path.join(data_file_path, "virus-lombardia.csv")
lomb_tmp_data_file = os.path.join(data_file_path, "virus-lombardia-{dt}.csv".format(dt=now.strftime("%Y%m%d")))



In [3]:
#result = requests.get(url_province_latest)
#if result.status_code not in ok_statuses:
#    assert "Get data failed. Received error code: {er}".format(er=str(result.status_code))
    
#with codecs.open(prov_data_file, "w", "utf-8") as fh:
#    fh.write(result.text)


In [4]:
#----------------------------------------------------------------
#
#----------------------------------------------------------------

def get_web_file(url):
    """
    
    :param url: 
    :return (rv, content):
    """
    rv = False
    result = None
    try:
        result = requests.get(url_region_pdf)
        if result.status_code not in ok_statuses:
            print("Get data failed. Received error code: {er}".format(er=str(result.status_code)))
        else:
            result = result.content
    except Exception as ex:
        print("get_web_file failed - {ex}".format(ex=ex))
    else:
        rv = True
    return (rv, result)    
        
def save_content_to_file(file_name, content):
    """
    
    :param file_name: 
    :return rv:
    """
    rv = False
    try:
        with open(file_name, "wb") as fh:
            fh.write(content)
    except Exception as ex:
        print("save_content_to_file failed - {ex}".format(ex=ex))
    else:
        rv = True
    return rv
   
def pdf_to_dataframe(pdf_file_name, version = "v2"):
    """
    
    :param version: valid values v1 or v2; 
    :param pdf_file_name: 
    :return rv:
    """
    rv = False
    df = None
    try:
        df = tabula.read_pdf(pdf_file_name, pages='all')
        #print("Df list len: {l}".format(l=len(df)))
        
        if version == "v2":
            csv_file = os.path.splitext(pdf_file_name)[0] + ".csv"
            tabula.convert_into(pdf_file_name, csv_file, output_format="csv", pages='all')
            list_reg = [] 
            with open(csv_file, "r") as fh:
                start = False
                end = False
                reg = re.compile("(\d{1,3}) (\d)")
                for line in fh:
                    if line.startswith("Lombardia") == True:
                        start = True
                    if line.startswith("TOTALE") == True:
                        end = True
                        start = False
                    if start == True:
                        line = line.replace(".", "")
                        line = line.replace("+ ", "")
                        #line = line.replace(" ", ",")
                        line = reg.sub("\\1,\\2", line)
                        line = line.replace("\n", "")
                        list_reg.append(line)
            #print("{lst}".format(lst=list_reg))
            df[0] = pd.DataFrame([line.split(",") for line in list_reg])

    except Exception as ex:
        print("pdf_to_dataframe failed - {ex}".format(ex=ex))
    else:
        rv = True
    return (rv, df)

def get_row_from_Italy_df(df):
    """
    :param df: 
    :return (rv, content):
    """
    rv = False
    csv_row = []
    try:
        csv_row.append(sample_date)
        
        infected_label = df.columns.values[0]
        infected_num = locale.atoi(df.columns.values[1])
        csv_row.append(str(infected_num))
        
        df.set_index(["ATTUALMENTE POSITIVI"], inplace=True)
        df["Totals"] = df[df.columns[0]].apply(lambda row: int(row) if row.is_integer() else int(row * 1000) )
        
        tot_deads = df.loc["TOTALE DECEDUTI", ['Totals']]
        tot_deads = str(int(tot_deads.values[0]))
        csv_row.append(tot_deads)
        
        tot_recovered = df.loc["TOTALE GUARITI", ['Totals']]
        tot_recovered = str(int(tot_recovered.values[0]))
        csv_row.append(tot_recovered)
        
        tot_infected = df.loc["CASI TOTALI", ['Totals']]
        tot_infected = str(int(tot_infected.values[0]))
        csv_row.append(tot_infected)
        
        csv_row = ",".join(csv_row)
        
    except Exception as ex:
        print("get_row_from_Italy_df failed - {ex}".format(ex=ex))
    else:
        rv = True
    return (rv, csv_row)

def append_row(file_name, row):
    """
    
    :param file_name:
    :param row:
    """
    rv = False
    try:
        with open(file_name, 'a') as fh:
            row = row + "\n"
            fh.write(row)
    except Exception as ex:
        print("append_row failed - {ex}".format(ex=ex))
    else:
        rv = True
    return rv

def refactor_region_df(df, version="v2"):
    """
    
    :param df: a list vor v2 or a dataframe for v1
    :param version: valid values v1 or v2; 
    :return (rv, df_region):
    """
    rv = False
    df_res = None
    try:
        if version == "v1":
            df_res = df.drop([0,1])
            df_res.rename(columns={df.columns[0]: "Regione"
                                  ,df.columns[1]: "Ricoverati con sintomi"
                                  ,df.columns[2]: "Terapia intensiva"
                                  ,df.columns[3]: "Isolamento domiciliare"
                                  ,df.columns[4]: "Totale attualmente positivi"
                                  ,df.columns[5]: "DIMESSI/GUARITI"
                                  ,df.columns[6]: "DECEDUTI"
                                  ,df.columns[7]: "CASI TOTALI"
                                  ,df.columns[8]: "TAMPONI"
                                  },
                         inplace = True)

            for idx in range(1, 6):
                df_res[df_res.columns[idx]] = df_res[df_res.columns[idx]].apply(lambda row: row.replace(".", "") )
            for idx in range(6, 9):
                df_res[df_res.columns[idx]] = df_res[df_res.columns[idx]].apply(lambda row: str(int(row)) if row.is_integer() else str(int(row * 1000)) )

        elif version == "v2":
            df_res = df
            df_res.rename(columns={df_res.columns[0]: "Regione"
                                  ,df_res.columns[1]: "Ricoverati con sintomi"
                                  ,df_res.columns[2]: "Terapia intensiva"
                                  ,df_res.columns[3]: "Isolamento domiciliare"
                                  ,df_res.columns[4]: "Totale attualmente positivi"
                                  ,df_res.columns[5]: "DIMESSI/GUARITI"
                                  ,df_res.columns[6]: "DECEDUTI"
                                  ,df_res.columns[7]: "INCREMENTO CASI TOTALI"
                                  ,df_res.columns[8]: "CASI TOTALI"
                                  ,df_res.columns[9]: "TAMPONI"
                              },
                          inplace = True)
        else:
            msg = "Unknown \"${v}\" version number.".format(v=version)
            print(msg)
            raise Exception(msg)

    except Exception as ex:
        print("refactor_region_df failed - {ex}".format(ex=ex))
    else:
        rv = True  
        df_res.set_index("Regione", inplace=True)
    print("rv -> {rv}".format(rv=rv))
    return (rv, df_res)

def get_region_row(df, region_name):
    """
    
    :param df: 
    :param region_name: 
    :return (rv, row):
    """
    rv = False
    row = None
    try:
        csv_row = []
        csv_row.append(sample_date)
        
        value = df.loc[region_name]['Ricoverati con sintomi']
        csv_row.append(value)
        
        value = df.loc[region_name]['Terapia intensiva']
        csv_row.append(value)

        value = df.loc[region_name]['Isolamento domiciliare']
        csv_row.append(value)

        value = df.loc[region_name]['Totale attualmente positivi']
        csv_row.append(value)

        value = df.loc[region_name]['DIMESSI/GUARITI']
        csv_row.append(value)
        
        value = df.loc[region_name]['DECEDUTI']
        csv_row.append(value)
        
        value = df.loc[region_name]['CASI TOTALI']
        csv_row.append(value)

        value = df.loc[region_name]['TAMPONI']
        csv_row.append(value)        
        
        row = ",".join(csv_row)
        
    except Exception as ex:
        print("get_region_row failed - {ex}".format(ex=ex))
    else:
        rv = True  
    return (rv, row)


In [5]:
#----------------------------------------------------------------
# Download the new dataset and append the new data in the csv
# files.
#
# Pay attention to the date: it is calculated as "now" so if the
# data aren't of today you shoud change the value by hand.
#----------------------------------------------------------------

df_regions = None
df_Italy = None

rv = False
result = get_web_file(url_region_pdf)
if result[0] == True:
    rv = save_content_to_file(pdf_file_name, result[1])
else:
    assert False, "File download failure."
    
if rv == True:
    df_list = pdf_to_dataframe(pdf_file_name)
if df_list[0] == True:
    df_regions = df_list[1][0]
    df_Italy = df_list[1][1]
else:
    assert False, "Unable to download or save the data file."

ita_row = get_row_from_Italy_df(df_Italy)    
if ita_row[0] == True:
    with open(it_data_file, 'r') as src, open(it_tmp_data_file, 'w') as dst: 
            dst.write(src.read())
            dst.close()
            src.close()
    rv = append_row(it_tmp_data_file, ita_row[1])

row_lomb = None
df_regions = refactor_region_df(df_regions, version="v2")
if df_regions[0] == True:
    df_regions = df_regions[1]
    row_lomb = get_region_row(df_regions, "Lombardia")
    if row_lomb[0] == False:
        assert False, "Region dataframe refactoring failed."
else:
    assert False, "Cannot create regions dataframe."
    
with open(lomb_data_file, 'r') as src, open(lomb_tmp_data_file, 'w') as dst: 
    dst.write(src.read())
    dst.close()
    src.close()
    rv = append_row(lomb_tmp_data_file, row_lomb[1])
    
print("Done")    

Got stderr: May 04, 2020 7:03:18 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font CIDFont+F1 are not implemented in PDFBox and will be ignored
May 04, 2020 7:03:18 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font CIDFont+F2 are not implemented in PDFBox and will be ignored
May 04, 2020 7:03:18 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font CIDFont+F3 are not implemented in PDFBox and will be ignored

Got stderr: May 04, 2020 7:03:20 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font CIDFont+F1 are not implemented in PDFBox and will be ignored
May 04, 2020 7:03:20 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font CIDFont+F2 are not implemented in PDFBox and will be ignored
May 04, 2020 7:03:20 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init

rv -> True
Done


In [6]:
df_regions

,Ricoverati con sintomi,Terapia intensiva,Isolamento domiciliare,Totale attualmente positivi,DIMESSI/GUARITI,DECEDUTI,INCREMENTO CASI TOTALI,CASI TOTALI,TAMPONI,10
Regione,,,,,,,,,,
Lombardia,6414,532,30361,37307,26504,14294,78105,577,418835,251661
Piemonte,2391,161,13010,15562,8874,3186,27622,192,176078,123801
Emilia Romagna,1968,199,6817,8984,13525,3666,26175,159,200427,133329
Veneto,955,101,6178,7234,9611,1528,18373,55,383660,223968
Toscana,512,107,4660,5279,3441,881,9601,38,152447,115065
Liguria,606,72,2830,3508,3683,1221,8412,53,56092,35443
Lazio,1335,95,2955,4385,1938,524,6847,38,153293,120095
Marche,389,44,2773,3206,2225,932,6363,44,65398,42838
Campania,438,24,2249,2711,1421,366,4498,14,90543,48275
